In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import itertools

In [ ]:
choices = range(10)

combinations = itertools.product(choices, repeat=5)

numbers = []

for c in combinations:
    number = ''.join(str(x) for x in c)
    numbers.append(number)

ids = []

for n in ['1', '2', '8', '9']:
    for number in numbers:
        id = n + number
        ids.append(id)

In [ ]:
indiana = pd.DataFrame(columns=[])

In [ ]:
for id in ids:
    url = 'https://www.in.gov/apps/indcorrection/ofs/ofs?previous_page=1&detail=' + id
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    fac = soup.find('strong', text = 'Facility/Location').find_parent('td').find_next_sibling('td').get_text()
    if fac not in ['', 'Discharge', 'Returned to court authority on release', 'Delinquent', 'Deceased']:
        table = soup.find('table', {'border': '0', 'width': '100%', 'bgcolor': '#FFFFFF'}).find_all('td')
        name = table[0].text
        dob = soup.find('strong', text = 'Date \n              of Birth').find_parent('td').find_next_sibling('td').get_text()
        gender = soup.find('strong', text = 'Gender').find_parent('td').find_next_sibling('td').get_text()
        race = soup.find('strong', text = 'Race').find_parent('td').find_next_sibling('td').get_text()

        sendates = []
        offenses = []
        sentences = []
        counties = []
        codes = []

        sendate_elements = soup.find_all('strong', text = 'Date \n              of Sentence')
        for sendate in sendate_elements:
            sendates.append(sendate.find_parent('td').find_next_sibling('td').get_text())
        offense_elements = soup.find_all('strong', text = 'Description')
        for offense in offense_elements:
            offenses.append(offense.find_parent('td').find_next_sibling('td').get_text())
        sentence_elements = soup.find_all('strong', text = 'Term \n              in  Years / Months / Days')
        for sentence in sentence_elements:
            sentables = sentence.find_parent('td').find_next_sibling('td').find_all('table')
            for sentable in sentables:
                times = sentable.find_all('td')
                years = times[0].text
                months = times[1].text
                days = times[2].text
                term = years + '-' + months + '-' + days
                sentences.append(term)
        county_elements = soup.find_all('strong', text = 'County \n              of Conviction')
        for county in county_elements:
            counties.append(county.find_parent('td').find_next_sibling('td').get_text())
        code_elements = soup.find_all('strong', text = 'Indiana \n              Citation Code')
        for code in code_elements:
            codes.append(code.find_parent('td').find_next_sibling('td').get_text())

        crimes = [list(pair) for pair in zip(offenses, sentences, sendates, counties, codes)]
        rows = []
        for pair in crimes:
            row = {
                'ID': id,
                'Name': name,
                'Date Of Birth': dob,
                'Gender': gender,
                'Race': race,
                'Facility': fac,
                'Offense Description': pair[0],
                'Sentence Length': pair[1],
                'Sentence Date': pair[2],
                'County': pair[3],
                'Offense Code': pair[4]
            }
            rows.append(row)
            
        indiana = pd.concat([indiana, pd.DataFrame(rows)])
